# STC Jawwy

In [2]:
"""
Here we install libraries that are not installed by default 
Example:  pyslsb
Feel free to add any library you are planning to use.
"""
!pip install pyxlsb

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
# Import the required libraries 
"""
Please feel free to import any required libraries as per your needs
"""
import pandas as pd     # provides high-performance, easy to use structures and data analysis tools
import pyxlsb           # Excel extention to read xlsb files (the input file)
import numpy as np      # provides fast mathematical computation on arrays and matrices

# Jawwy dataset
The dataset consists of details about each customer and the movies and/or tv shows watched in addition to the genre. 

You are required to work on task three to build a recommendation engine for our platform to Recommend movies to usesrs that they might be interested in¶


In [4]:
# get the data
dataframe = pd.read_excel("stc TV Data Set_T3.xlsx",index_col=0)

# check the data shape
dataframe.shape

(1048575, 5)

In [5]:
# display the first 5 rows 
dataframe.head()

,user_id_maped,program_name,rating,date_,program_genre
0,26138,100 treets,1,2017-05-27,Drama
1,7946,Moana,1,2017-05-21,Animation
2,7418,The Mermaid Princess,1,2017-08-10,Animation
3,19307,The Mermaid Princess,2,2017-07-26,Animation
4,15860,Churchill,2,2017-07-07,Biography


In [6]:
# describe the numeric values in the dataset
dataframe.describe()

,user_id_maped,rating,date_
count,1.048575e+06,1.048575e+06,1048575
mean,1.709266e+04,2.497283e+00,2017-10-04 00:23:20.346183936
min,1.000000e+00,1.000000e+00,2017-03-14 00:00:00
25%,8.253000e+03,1.000000e+00,2017-06-10 00:00:00
50%,1.714900e+04,2.000000e+00,2017-10-14 00:00:00
75%,2.566500e+04,3.000000e+00,2018-01-21 00:00:00
max,3.428000e+04,4.000000e+00,2018-04-30 00:00:00
std,1.003513e+04,1.119837e+00,NaN


In [7]:
# check if any column has null value in the dataset
dataframe.isnull().any()

user_id_maped    False
program_name     False
rating           False
date_            False
program_genre    False
dtype: bool

In [8]:
# we import Visualization libraries 
# you can ignore and use any other graphing libraries 
import matplotlib.pyplot as plt # a comprehensive library for creating static, animated, and interactive visualizations
import plotly #a graphing library makes interactive, publication-quality graphs. Examples of how to make line plots, scatter plots, area charts, bar charts, error bars, box plots, histograms, heatmaps, subplots, multiple-axes, polar charts, and bubble charts.
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [9]:
# build Recommender system to Highlight Programs that usesrs might be interested in
# copy dataset
df = dataframe.copy()

# Create user-item matrix
user_program_matrix = df.pivot_table(index='user_id_maped', columns='program_name', values='rating')

# Fill NaNs with 0 for similarity calculation
user_program_matrix_filled = user_program_matrix.fillna(0)




In [10]:
# Build User Similarity Matrix
from sklearn.metrics.pairwise import cosine_similarity

# Compute user-user similarity
user_similarity = cosine_similarity(user_program_matrix_filled)
user_similarity_df = pd.DataFrame(user_similarity, index=user_program_matrix.index, columns=user_program_matrix.index)


In [11]:
# recommendations (top 5) for the people who watched "Moana" movie
# Find users who watched Moana
moana_viewers = df[df['program_name'] == 'Moana']['user_id_maped'].unique()

# Store recommendations
recommendations = {}

for viewer in moana_viewers:
    # Get similar users
    similar_users = user_similarity_df[viewer].sort_values(ascending=False)[1:6]  # top 5 similar users

    # Get programs those similar users rated
    similar_users_ids = similar_users.index
    similar_users_data = df[df['user_id_maped'].isin(similar_users_ids)]

    # Aggregate and rank programs not yet watched by the current viewer
    viewer_programs = df[df['user_id_maped'] == viewer]['program_name'].unique()
    recommended_programs = (
        similar_users_data[~similar_users_data['program_name'].isin(viewer_programs)]
        .groupby('program_name')['rating']
        .mean()
        .sort_values(ascending=False)
        .head(5)
    )

    recommendations[viewer] = recommended_programs

# Display recommendations for the first Moana viewer
moana_first = moana_viewers[0]
print(f"Top 5 Recommendations for user {moana_first} (who watched Moana):")
print(recommendations[moana_first])

Top 5 Recommendations for user 7946 (who watched Moana):
program_name
Surf's Up : WaveMania    2.000000
The Birth of a Nation    2.000000
Alien: Covenant          1.989130
The Boss Baby            1.333333
100 treets               1.000000
Name: rating, dtype: float64


A collaborative filtering model was built to recommend programs based on similar users’ viewing behavior.
For users who watched “Moana”, the system suggested 5 top-rated programs including Surf’s Up: WaveMania and Alien: Covenant, enhancing the personalization experience on STC TV.